# Face Recognition Attendance System

This notebook includes:
1. Configuration
2. Loading dataset and encoding faces
3. Training KNN classifier
4. Calculating distance threshold
5. Saving model
6. Real-time webcam recognition
7. Optional: live attendance dashboard

In [1]:
# --- CONFIGURATION ---
class Config:
    CAMERA_INDEX = 0
    MODEL_PATH = "model.pkl"
    FACE_DETECT_MODEL = "hog"  # "hog" (fast) or "cnn" (accurate, GPU only)
    FRAME_WIDTH = 640
    PROCESS_EVERY_N = 3
    DISTANCE_THRESHOLD = None  # previously computed
    ENCODING_NORMALIZE = True  # L2 normalize encodings for stability
    SSE_UPDATE_INTERVAL = 1  # dashboard update frequency
    DEBOUNCE_SECONDS = 3  # seconds to debounce IN/OUT toggles

In [2]:
# --- IMPORTS ---
import os
import pandas as pd
import numpy as np
import cv2
import threading
import face_recognition
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# --- HELPER FUNCTIONS ---
def normalize(vec):
    norm = np.linalg.norm(vec)
    return vec / norm if norm > 0 else vec

In [4]:
# --- LOAD DATA AND ENCODE FACES ---
labels_df = pd.read_csv("dataset/Dataset.csv")
encodings, labels = [], []

for idx, row in labels_df.iterrows():
    img_path = os.path.join("dataset/Faces/", row["id"])
    if not os.path.exists(img_path):
        continue
    print(f"Processing image: {idx + 1} / {len(labels_df)}")
    image = face_recognition.load_image_file(img_path)
    face_locations = face_recognition.face_locations(image, model=Config.FACE_DETECT_MODEL)
    if not face_locations:
        continue
    encoding = face_recognition.face_encodings(image, face_locations)[0]
    if Config.ENCODING_NORMALIZE:
        encoding = normalize(encoding)
    encodings.append(encoding)
    labels.append(row["label"])

if not encodings:
    raise RuntimeError("No encodings found. Training aborted.")

X = np.array(encodings)
y = np.array(labels)

Processing image: 1 / 2562
Processing image: 2 / 2562
Processing image: 3 / 2562
Processing image: 4 / 2562
Processing image: 5 / 2562
Processing image: 6 / 2562
Processing image: 7 / 2562
Processing image: 8 / 2562
Processing image: 9 / 2562
Processing image: 10 / 2562
Processing image: 11 / 2562
Processing image: 12 / 2562
Processing image: 13 / 2562
Processing image: 14 / 2562
Processing image: 15 / 2562
Processing image: 16 / 2562
Processing image: 17 / 2562
Processing image: 18 / 2562
Processing image: 19 / 2562
Processing image: 20 / 2562
Processing image: 21 / 2562
Processing image: 22 / 2562
Processing image: 23 / 2562
Processing image: 24 / 2562
Processing image: 25 / 2562
Processing image: 26 / 2562
Processing image: 27 / 2562
Processing image: 28 / 2562
Processing image: 29 / 2562
Processing image: 30 / 2562
Processing image: 31 / 2562
Processing image: 32 / 2562
Processing image: 33 / 2562
Processing image: 34 / 2562
Processing image: 35 / 2562
Processing image: 36 / 2562
P

In [5]:
# --- TRAIN KNN CLASSIFIER ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

knn = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
knn.fit(X_scaled, y)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'euclidean'
,metric_params,None
,n_jobs,None


In [6]:
# --- CALCULATE DISTANCE THRESHOLD ---
distances, _ = knn.kneighbors(X_scaled, n_neighbors=2)
nearest_distances = distances[:, 1]  # nearest other face
threshold = nearest_distances.mean() + 2 * nearest_distances.std()
print(f"Recommended DISTANCE_THRESHOLD = {threshold:.3f}")
Config.DISTANCE_THRESHOLD = threshold

Recommended DISTANCE_THRESHOLD = 9.642


In [7]:
# --- SAVE MODEL ---
joblib.dump({
    "scaler": scaler,
    "model": knn,
}, Config.MODEL_PATH)
print(f"Model saved to {Config.MODEL_PATH}")

Model saved to model.pkl


# --- REAL-TIME WEBCAM RECOGNITION ---

In [8]:
# Attendance helpers
from datetime import datetime, timedelta
attendance_status = {}  # name -> in/out
last_event_time = {}

def mark_seen(name):
    now = datetime.now()
    last_time = last_event_time.get(name)
    if last_time is not None and now - last_time < timedelta(seconds=Config.DEBOUNCE_SECONDS):
        return
    prev = attendance_status.get(name)
    attendance_status[name] = 'in' if prev != 'in' else 'out'
    last_event_time[name] = now

In [9]:
# Recognition helpers
def get_encodings(rgb_image):
    boxes = face_recognition.face_locations(rgb_image, model=Config.FACE_DETECT_MODEL)
    if not boxes:
        return [], []
    encs = face_recognition.face_encodings(rgb_image, boxes)
    if Config.ENCODING_NORMALIZE:
        encs = [normalize(e) for e in encs]
    return encs, boxes

def predict(pipeline, encoding):
    X_input = np.array(encoding).reshape(1, -1)
    X_scaled = pipeline["scaler"].transform(X_input)
    distances, _ = pipeline["model"].kneighbors(X_scaled, n_neighbors=1)
    closest_dist = distances[0][0]
    label = pipeline["model"].predict(X_scaled)[0]
    return label if closest_dist <= Config.DISTANCE_THRESHOLD else "Unknown"

In [ ]:
from flask import Flask, render_template, Response, stream_with_context
import time
import json

from attendance import get_events, get_status, get_event_id
from config import Config

app = Flask(__name__)


@app.route("/")
def index():
    return render_template(
        "attendance.html",
        events=get_events(),
        status=get_status(),
    )


@app.route("/events")
def sse_stream():
    """
    SSE endpoint:
    - updates ONLY when a new attendance event is created (IN/OUT)
    - extremely fast (100ms check)
    - no unnecessary polling
    """

    def stream():
        last_id = get_event_id()

        while True:
            current_id = get_event_id()

            if current_id != last_id:
                payload = {"events": get_events(), "status": get_status()}

                last_id = current_id

                yield f"event: update\ndata: {json.dumps(payload)}\n\n"

            time.sleep(0.1)  # 100ms → near-instant response

    response = Response(stream_with_context(stream()), mimetype="text/event-stream")

    # Explicit SSE headers (prevent buffering)
    response.headers["Cache-Control"] = "no-cache"
    response.headers["X-Accel-Buffering"] = "no"
    response.headers["Connection"] = "keep-alive"

    return response


def run_dashboard():
    app.run(host="0.0.0.0", port=5000, threaded=True, use_reloader=False)


if __name__ == "__main__":
    run_dashboard()


In [12]:
# --- RUN REAL-TIME WEBCAM ---
pipeline = joblib.load(Config.MODEL_PATH)

def start_dashboard():
    t = threading.Thread(target=run_dashboard, daemon=True)
    t.start()
    print("Dashboard running @ http://127.0.0.1:5000")

cap = cv2.VideoCapture(Config.CAMERA_INDEX)
frame_count = 0
last_detections = []

print("Press Q to quit")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    original = frame.copy()
    h, w = frame.shape[:2]
    scale = Config.FRAME_WIDTH / w if w > Config.FRAME_WIDTH else 1.0
    frame_small = cv2.resize(frame, (0,0), fx=scale, fy=scale)
    rgb_small = cv2.cvtColor(frame_small, cv2.COLOR_BGR2RGB)

    if frame_count % Config.PROCESS_EVERY_N == 0:
        encs, boxes = get_encodings(rgb_small)
        detections = []
        for enc, (top,right,bottom,left) in zip(encs, boxes):
            name = predict(pipeline, enc)
            t,r,b,l = int(top/scale), int(right/scale), int(bottom/scale), int(left/scale)
            detections.append((name,(t,r,b,l)))
            if name != "Unknown":
                mark_seen(name)
        last_detections = detections.copy()
    else:
        detections = last_detections

    for name, (t,r,b,l) in detections:
        color = (0,255,0) if name != "Unknown" else (0,0,255)
        cv2.rectangle(original,(l,t),(r,b),color,2)
        cv2.putText(original,name,(l,t-8),cv2.FONT_HERSHEY_SIMPLEX,0.7,color,2)

    cv2.imshow("Face Attendance", original)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    frame_count += 1

cap.release()
cv2.destroyAllWindows()
print("Attendance Status:", attendance_status)

Press Q to quit
Attendance Status: {np.str_('Priyanka Chopra'): 'in'}
